In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
def cgi_decode(input_data):
    """Decodes the CGI-encoded string "input_data":
       - Replaces "+" by " ".
       - Replaces "%xx" by the character with hex number xx.
       Returns the decoded string.  Raises `ValueError` for invalid input_datas.
       """
    output = ""
    i = 0
    while i < len(input_data):
        current_char = input_data[i]
        if current_char == "+":
            output += " "
        elif current_char == "%":
            digit_high, digit_low = input_data[i + 1], input_data[i + 2]
            i += 2
            try:
                v = int(digit_high, 16) * 16 + int(digit_low, 16)
                output += chr(v)
            except:
                raise ValueError("Invalid input_data")
        else:
            output += current_char
        i += 1
    return output

<IPython.core.display.Javascript object>

In [3]:
def line_tracer(frame, event, arg):
    """Traces the collection of lines executed by a function."""
    if event == "line":
        lineno = frame.f_lineno
        global coverage
        coverage.add(lineno)
    return line_tracer

<IPython.core.display.Javascript object>

In [4]:
import sys


def record_coverage(function, s):
    """Record the set of lines covered when function calls s."""
    global coverage
    coverage = set([])
    sys.settrace(line_tracer)
    function(s)
    sys.settrace(None)
    coverage = frozenset(coverage)
    return coverage

<IPython.core.display.Javascript object>

In [5]:
class Individual:
    def __init__(self, value):
        self.value = value
        try:
            self.coverage = record_coverage(cgi_decode, self.value)
        except:
            self.coverage = str(sys.exc_info()[0])
            print("Got error " + str(sys.exc_info()[0]) + " for input " + self.value)
        add_to_coverage_dict(self.value, self.coverage)
        self.fitness = None

    def print_info(self):
        print(self.value)
        print(self.coverage)
        print(self.fitness)

    def compute_fitness(self):
        self.fitness = 1 / (coverage_dict[self.coverage])

<IPython.core.display.Javascript object>

In [6]:
important_samples = []
coverage_dict = {}


def add_to_coverage_dict(value, coverage):
    if coverage_dict.get(coverage, 0) == 0:
        important_samples.append(value)
    coverage_dict[coverage] = coverage_dict.get(coverage, 0) + 1

<IPython.core.display.Javascript object>

In [7]:
import numpy as np

carrying_capacity = 100
seed = [
    "http://www.google.com/search?q=fuzzing",
    "http://www.google.com/searh;q=fuzzinb2g",
    "http://www.Xgole.com/eaHrh;qfu/zzinb2g",
]
population = [Individual(x) for x in seed]

<IPython.core.display.Javascript object>

In [8]:
def update_fitness_scores(population):
    for individual in population:
        individual.compute_fitness()

<IPython.core.display.Javascript object>

In [9]:
def print_population_info(population):
    for individual in population:
        individual.print_info()

<IPython.core.display.Javascript object>

In [10]:
import numpy as np


def get_fitness_scores(population):
    fitness_distribution = np.array([x.fitness for x in population])
    fitness_distribution = fitness_distribution / sum(fitness_distribution)
    return fitness_distribution

<IPython.core.display.Javascript object>

In [11]:
import numpy as np


def sample_pair_for_reproduction(population):
    fitness_distribution = get_fitness_scores(population)
    index1 = np.random.choice(len(fitness_distribution), p=fitness_distribution)
    individual1 = population[index1]
    index2 = np.random.choice(len(fitness_distribution), p=fitness_distribution)
    individual2 = population[index2]
    return individual1, individual2

<IPython.core.display.Javascript object>

In [12]:
def recombine(individual1, individual2):
    L1 = len(individual1.value)
    L2 = len(individual2.value)
    L = min(L1, L2)
    locus = np.random.randint(L)
    offspring1_value = individual1.value[:locus] + individual2.value[locus:]
    offspring2_value = individual2.value[:locus] + individual1.value[locus:]
    return Individual(offspring1_value), Individual(offspring2_value)

<IPython.core.display.Javascript object>

In [13]:
update_fitness_scores(population)

<IPython.core.display.Javascript object>

In [14]:
print_population_info(population)

http://www.google.com/search?q=fuzzing
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.3333333333333333
http://www.google.com/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.3333333333333333
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.3333333333333333


<IPython.core.display.Javascript object>

In [15]:
important_samples

['http://www.google.com/search?q=fuzzing']

<IPython.core.display.Javascript object>

In [16]:
def recombination_phase(population):
    new_population = population.copy()
    population_size = len(population)
    number_of_recombinations = population_size // 2
    for i in range(number_of_recombinations):
        individual1, individual2 = sample_pair_for_reproduction(population)
        try:
            offspring1, offspring2 = recombine(individual1, individual2)
            new_population.append(offspring1)
            new_population.append(offspring2)
        except ValueError:
            pass
    update_fitness_scores(new_population)
    return new_population

<IPython.core.display.Javascript object>

In [17]:
population = recombination_phase(population)

<IPython.core.display.Javascript object>

In [18]:
print_population_info(population)

http://www.google.com/search?q=fuzzing
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.google.com/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.google.cm/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.coom/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2


<IPython.core.display.Javascript object>

In [19]:
MUTATION_PROBABILITY = 0.1
import random
import string


def produce_random_character():
    vocabulary = string.punctuation + string.ascii_letters + string.digits
    return random.choice(vocabulary)


def delete_random_character(input_data):
    """Randomly deletes a character in the in file."""
    l = len(input_data)
    position = random.randint(0, l - 1)
    output = input_data[:position] + input_data[(position + 1) :]
    return output


def insert_random_character(input_data):
    """Randomly inserts a character into the in file."""
    output = input_data
    l = len(input_data)
    position = random.randint(0, l - 1)
    inserted_character = produce_random_character()
    output = input_data[:position] + inserted_character + input_data[position:]
    return output


def swap_two_characters(input_data):
    """Swap two characters at random."""
    l = len(input_data)
    position1 = random.randint(0, l - 1)
    character1 = input_data[position1]
    position2 = random.randint(0, l - 1)
    character2 = input_data[position2]
    if position1 == position2:
        return input_data
    min_position = min(position1, position2)
    max_position = max(position1, position2)
    output = (
        input_data[:min_position]
        + input_data[max_position]
        + input_data[(min_position + 1) : (max_position)]
        + input_data[min_position]
        + input_data[(max_position + 1) :]
    )
    return output


def randomize_character(input_data):
    """Randomly changes a character in the in file."""
    l = len(input_data)
    position = random.randint(0, l - 1)
    character_after = produce_random_character()
    output = input_data[:position] + character_after + input_data[(position + 1) :]
    return output


def mutate(input_data, num_mutations=3):
    """Perform num_mutations many mutations on in."""
    output = input_data
    mutation_operations = [
        randomize_character,
        insert_random_character,
        delete_random_character,
        swap_two_characters,
    ]
    for i in range(num_mutations):
        current_mutation_operation = random.choice(mutation_operations)
        output = current_mutation_operation(output)
    return output


def mutation_phase(population):
    for i in range(len(population)):
        individual = population[i]
        mutate_this_individual = np.random.choice(
            2, p=[1 - MUTATION_PROBABILITY, MUTATION_PROBABILITY]
        )
        if mutate_this_individual:
            try:
                mutated_individual = Individual(mutate(individual.value))
                population[i] = mutated_individual
            except ValueError:
                pass
    update_fitness_scores(population)
    return population

<IPython.core.display.Javascript object>

In [20]:
population = mutation_phase(population)

<IPython.core.display.Javascript object>

In [21]:
print_population_info(population)

http://www.google.com/search?q=fuzzing
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.google.com/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.google.cm/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.coom/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2


<IPython.core.display.Javascript object>

In [22]:
def culling_phase(population):
    population_size = len(population)
    N = min(carrying_capacity, population_size)
    fitness_distribution = get_fitness_scores(population)
    new_population = list(np.random.choice(population, N, p=fitness_distribution))
    update_fitness_scores(new_population)
    return new_population

<IPython.core.display.Javascript object>

In [23]:
population = culling_phase(population)

<IPython.core.display.Javascript object>

In [24]:
print_population_info(population)

http://www.Xgole.coom/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.google.com/searh;q=fuzzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2
http://www.Xgole.com/eaHrh;qfu/zzinb2g
frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24})
0.2


<IPython.core.display.Javascript object>

In [25]:
number_of_generations = 50
for g in range(number_of_generations):
    print("generation " + str(g))
    population = recombination_phase(population)
    population = mutation_phase(population)
    population = culling_phase(population)

generation 0
generation 1
generation 2
generation 3
generation 4
generation 5
generation 6
generation 7
generation 8
Got error <class 'ValueError'> for input http:/%wwwXgol.Hcom/ea.rh;qfu}uzzinb>2g
Got error <class 'ValueError'> for input http://w.w.Xgole.com%/eseahh;`q=fuzzin#b2g
generation 9
Got error <class 'ValueError'> for input http:/%wwwXgol.Hcom/eam/searh;q=fuzznb22g
Got error <class 'ValueError'> for input http:/%wwwXgol.Hcom/ea.rh;qfu}uzzizznb22g
Got error <class 'ValueError'> for input http:/%wwwXgol.Hcom/ea.seahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http://w.w.Xgole.com%/erh;qfu}uzzinb>2g
Got error <class 'ValueError'> for input http://w.w.Xgole.com%/eseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http://w.w.Xgole.com%/eseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http:/%wwwXgol.Hcom/ea.rh;qfu}uzzizznb22g
Got error <class 'ValueError'> for input http:/%wwwXgol.Hcom/ea.rh;qfu}uzzinb>2g
Got error <class 'ValueError'> for input h

Got error <class 'ValueError'> for input http:/%wwwXgol.Hcom/ea.rh;qh;qfufuzzib2g
Got error <class 'ValueError'> for input http://www.go`gcleom%/eseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http:/%wwwXgolv.Hcom/ea.rhqLfu}uzzinb>2g
Got error <class 'ValueError'> for input http:/%wwwXgol.Hgom/ea.h;qfu}uz,zinb>2c
Got error <class 'ValueError'> for input htfp\//w.%.Xgole.cnmw/esehh;`q=fuzziho#b2g
Got error <class 'ValueError'> for input httpg/%wwVwX2ol.Hcom/dea.rh;qfu}uzzizznb2Fg
Got error <class 'ValueError'> for input http:/%wwwX:gol.Hcom/ea.rh;fu}uzzinb>2hg
Got error <class 'ValueError'> for input http:.%wwwXgol/qcom/ea.rh;Hfu}uzzinb>g
Got error <class 'ValueError'> for input http:/%w;wXgol.com/e^a.rh;qfu}uzzinb>222g
Got error <class 'ValueError'> for input htfp://w.w.XPgole.com%/esehh;`quzzizznb22
Got error <class 'ValueError'> for input http:/%wwwXgol.Hceom/ea.rh;qfu}uzzinb>2
Got error <class 'ValueError'> for input ht#tp://.w.Xgole.eFm%/csZeahh;`q=fuzzin#b2g
Got erro

Got error <class 'ValueError'> for input http://zww.goole.com%/eseahh;`q@fuznb22g
generation 13
Got error <class 'ValueError'> for input http:/%w.w.Xgolecom/oom/searh;q=zinb>2g
Got error <class 'ValueError'> for input http:/%wwsw.+Xgo_e.ccom/weah;qfufuzzib2g
Got error <class 'ValueError'> for input ht#t4://ww.go`gcl.com%/es.ahh;`q=ifuzzin#b2g
Got error <class 'ValueError'> for input http:/%www.go`gcl.com%/es.ahu}uzzinb>222g
Got error <class 'ValueError'> for input http://wG..ogole.cXm%o.rh;qfh;`q=ifuzzin#b2g
Got error <class 'ValueError'> for input http:/%w.w.Xgole.com%/eseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input ht#tp/%w.w.Xgole.com%/eseahh;qfufuzzib2g
Got error <class 'ValueError'> for input http:/%wwwXgol.Hco.+oomseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http://w..Xgole.com%oom/searh;q=fuzzn2g
Got error <class 'ValueError'> for input ht#}:/%wwwXgol.Hcom/eam/searh;q=zinb>2g
Got error <class 'ValueError'> for input htfp\//w.%.Xgole.cnmw/esehh;`q=fu

Got error <class 'ValueError'> for input ht#p://www.go`gclcom%/eseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http:/%wwww.F+Xgole.cqom/s5eah;q=fuzznOCp22g
Got error <class 'ValueError'> for input http:/%w;wXXgole.com%/eseahh;`q=fuzzin22g
Got error <class 'ValueError'> for input http:/%w.w.gol.com/e^a.rh;qfu}u=fuzzin#b2g
Got error <class 'ValueError'> for input ht#tp:%w.w.<o`gcle.+ooum/searh;q=fuzznb22g
Got error <class 'ValueError'> for input http://w.w.Xgole.com%/eseahh;q;q=fuzznOCp22g
Got error <class 'ValueError'> for input http:/%wwwXgol.Hole.cqom/s5eahfufuzzib2g
Got error <class 'ValueError'> for input http:/%w;wXgol.com/e^/eseahh;;q=fuzznb22g
Got error <class 'ValueError'> for input htttp/%www.go`gcl.co%/eseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http://www.go`gclcom%/eseahh;;q=fuzznb22g
Got error <class 'ValueError'> for input htp:f/%w.ww.+Xgoe.com/searh;q=zinb>2g
Got error <class 'ValueError'> for input htttp:%w.w.Xgole.com%/eseahh;`q=fuzzin22g

Got error <class 'ValueError'> for input ht#p://www.go`gclcom%omseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input ht#p://www.go`gclcom%omseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http:/%/ww.Xgole.com%om/searh;q=uzzihn#b2g
Got error <class 'ValueError'> for input http:/%wwwX.+Xgcle.+ooesehh;`q=ffuzzin#b2g
Got error <class 'ValueError'> for input http:/wwww.go`gclcom%omseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http:/%wwwXgol.com/;^a#rheqfu}u=fnb22g
Got error <class 'ValueError'> for input http://wwg.Xgole.com%/esearh;q=zinb>2g
Got error <class 'ValueError'> for input http:/%w;wXXgol@e.+oom/se/4searh;q=fuzznb22g
Got error <class 'ValueError'> for input http:/%wwwXgol.com/;^a#rheqfu}u=fuzzzb2g
Got error <class 'ValueError'> for input httpp:/w.w.Xgole.com%/eseahh;`q=fuzzin#b2g
Got error <class 'ValueError'> for input http://www.go`gcle.m%/eseahh;q=fuzzihn#b2g
Got error <class 'ValueError'> for input ht#p://www.go`gclcom%omseahh;`q=fucuzznOCp2

...

Got error <class 'ValueError'> for input ht#tp:/wwww.g'le.com%g/rhearh;q=fuzznn#b2g
Got error <class 'ValueError'> for input ht#tt:T//www.go`gclm%oms>meahq=fuzznn22g
Got error <class 'ValueError'> for input ht#tp:/wwww.g'le.com%g#reahh;q=fuuzzin22g
Got error <class 'ValueError'> for input /tttp/%w.w.Xgole.m/;^a#rhahh;q=fuDzin22g
Got error <class 'ValueError'> for input ht#t//DwwXgn`gclen/;%g/esearh-=kufzio22g
Got error <class 'ValueError'> for input tt#://kwZXzo`gclecog%g/esea5enq=fuumzi;2g
Got error <class 'ValueError'> for input ht#t//kwwXgo`gcl.com%g/esea5enq=fuzzin2g
Got error <class 'ValueError'> for input ttttp/%w.w..ol.com/;^a(#reafh;q=zfzzin#b2g
Got error <class 'ValueError'> for input /tttp/%w.w..ol.cle.+oom/eqfu}u==fuzzin#b2g
Got error <class 'ValueError'> for input /tttp/%w.w..ol.com/;^a(#reahh;q=9uuzzin)2g
Got error <class 'ValueError'> for input ht#tp:/wwwwole.cm/oS%oms>me;q\=uinb>2g
Got error <class 'ValueError'> for input ttt/wwww.Xg.g'le.com%g#reahh;q=fuDzin22g
Got erro

Got error <class 'ValueError'> for input 2t#tt/wwww.Xgole.wcoS2oms>mehq=fuznn%h2g
Got error <class 'ValueError'> for input ht#tt/wwww.Xgole.moS2ocs>m?ehq=fu<znn%2g
Got error <class 'ValueError'> for input ht#t2/www.Xgole.coS2oms>meh=q=fu<znn%t2g
Got error <class 'ValueError'> for input ht#tt/wwww.Xgole.coS2omshm%e>q=fu<znT%22g
generation 37
Got error <class 'ValueError'> for input ht#tVt/wwww.Xgole.co2Som>mehq=fu<znn%l+
Got error <class 'ValueError'> for input ht#tt/wwwN.gXgole.coS2ms>mehqfu<znn%2g
Got error <class 'ValueError'> for input ht#tt/$2www.XKgole.coS2oms>mehq=fu<znn%w2g
Got error <class 'ValueError'> for input ht#tt/wwww.Xgole.co2Som%meq=fu<zn)>22+
Got error <class 'ValueError'> for input ht#Bt/wwww.Xgole.co2Smm>mehq=fu<znn%2o+
generation 38
Got error <class 'ValueError'> for input ht#tt/wwww.Xgole.coS2om>mehq=fu<znn%2o+
Got error <class 'ValueError'> for input ht#tt/w}ww.Xgol.co2Som>mehq%fu<znn=22+
Got error <class 'ValueError'> for input htBt/wwww.Xgole.co2Smm>qehq=fu<znn%

Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>meho=fu<znn%g
Got error <class 'IndexError'> for input h.#tt/wwww.XgoletcoqS2m>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#ttzwwwm.Xgole.co2Sow>mhq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input hn#ttqwwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>meh/=fuRztn%g
Got error <class 'IndexError'> for input ht#tt/wwww.X

Got error <class 'IndexError'> for input ht#ttnwwwwcXgole.co2Som>mehqfu<z/nn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<zn%g
Got error <class 'IndexError'> for input httttnwwwwcXgole.co2Som>mehqfu<z/n%g
Got error <class 'IndexError'> for input ht#/wwww.Xdole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#ttnwwwwcXgole.co2Som>mehqfu<z/n%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2Somm>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt.wwSw.Xgole/co!2wo>meh/=fuRztn%g
Got error <class 'IndexError'> for input htt/wwww.Xdole.co2Sodom>mehq=fu<zn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgo"e.co2m>mehq=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole..o2Som>eh=fu<znn%g
Got error <class 'IndexError'> for input ht#tt/wwwwcXgole.co2Som>meh/=fuRztn%g
Got error <class 'IndexError'> for input ht#tt/wwww.Xgole.co2

<IPython.core.display.Javascript object>

In [34]:
len(population)

100

<IPython.core.display.Javascript object>

In [35]:
important_samples

['http://www.google.com/search?q=fuzzing',
 'http:/%wwwXgol.Hcom/ea.rh;qfu}uzzinb>2g',
 'ht#tp://www.+Xgole.ccom/seah;qfufuzzib2g',
 'ht#tt/wwww.Xgole.coS2oms>mehq=fu<znn%22g',
 'ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%22+',
 'ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g']

<IPython.core.display.Javascript object>

In [36]:
cgi_decode("http://www.google.com/search?q=fuzzing")

'http://www.google.com/search?q=fuzzing'

<IPython.core.display.Javascript object>

In [37]:
cgi_decode("http:/%wwwXgol.Hcom/ea.rh;qfu}uzzinb>2g")

ValueError: Invalid input_data

<IPython.core.display.Javascript object>

In [38]:
cgi_decode("ht#tp://www.+Xgole.ccom/seah;qfufuzzib2g")

'ht#tp://www. Xgole.ccom/seah;qfufuzzib2g'

<IPython.core.display.Javascript object>

In [39]:
cgi_decode("ht#tt/wwww.Xgole.coS2oms>mehq=fu<znn%22g")

'ht#tt/wwww.Xgole.coS2oms>mehq=fu<znn"g'

<IPython.core.display.Javascript object>

In [40]:
cgi_decode("ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%22+")

'ht#tt/wwww.Xgole.co2Som>mehq=fu<znn" '

<IPython.core.display.Javascript object>

In [41]:
cgi_decode("ht#tt/wwww.Xgole.co2Som>mehq=fu<znn%g")

IndexError: string index out of range

<IPython.core.display.Javascript object>

In [42]:
coverage_dict

{frozenset({7, 8, 9, 10, 11, 13, 22, 23, 24}): 1297,
 "<class 'ValueError'>": 1274,
 frozenset({7, 8, 9, 10, 11, 12, 13, 22, 23, 24}): 1253,
 frozenset({7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 22, 23, 24}): 330,
 frozenset({7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 22, 23, 24}): 593,
 "<class 'IndexError'>": 793}

<IPython.core.display.Javascript object>